# Color Correction Tutorial 

In [ ]:
from plantcv import plantcv as pcv
import cv2
import numpy as np

In [ ]:
class options:
    def __init__(self):
        self.debug = "plot"
        self.writeimg= False
        self.result = "./color_tutorial_results"
        self.outdir = "."
        
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

In [ ]:
# Read in source and target images 

# Inputs:
#   filename - Image file to be read in 
#   mode - Return mode of image; either 'native' (default), 'rgb', or 'gray' 
target_img = pcv.readimage("img/tutorial_images/color_correct/target_img.png")
source_img = pcv.readimage("img/tutorial_images/color_correct/source1_img.png")

### Complete Color Card 

In [ ]:
# Create a labeled color card mask, first use color card finder function 

# Inputs:
#   rgb_img - RGB image data containing color card 
#   threshold - Optional threshold method; either 'adaptgauss' (default), 'normal', or 'otsu'
#   threshvalue - Optional threhsolding value (default threshvalue = 125) 
#   blurry - Optional boolean; False (default) or if True then image sharpening is applied 
#   background - Optional type of image background; 'dark' (default) or 'light'
dataframe1, start, space = pcv.transform.find_color_card(rgb_img=target_img, background='light')

In [ ]:
# Make the labeled mask 

# Inputs: 
#   rgb_img - RGB image data containing color card 
#   radius - Radius of color card chips (masks make circles on chips)
#   start_coord - Two-element tuple of the first chip mask, (starting x, starting y) 
#   spacing - Two-element tuple of the horizontal and vertical spacing between chip masks
#   nrows - Number of chip rows
#   ncols - Number of chip columns 
#   exclude - Optional list of chips to exclude. List largest to smallest index 
mask = pcv.transform.create_color_card_mask(target_img, radius=10, start_coord=start, spacing=space, nrows=6, ncols=4)

In [ ]:
# Our color card chip appears to be in the same position in both the source and target images 
# Try using the same parameters and check to make sure it's appropriate for the source image

source_mask = pcv.transform.create_color_card_mask(source_img, radius=10, start_coord=start, spacing=space, nrows=6, ncols=4)

In [ ]:
# Generate color transformation matrices 

# Inputs:
#   target_img - RGB image with color chips
#   target_mask - Grayscale image with color chips and background each represented with unique values 
#   source_img - RGB image with color chips 
#   source_mask - Grayscale image with color chips and background each represented with unique values 
#   output_directory - File path to which the target_matrix, source_matrix, and tranformation_matrix will be saved
tm, sm, z1_transformation_matrix, corrected_img = pcv.transform.correct_color(target_img=target_img, 
                                                                            target_mask=mask, 
                                                                            source_img=source_img, source_mask=mask, 
                                                                            output_directory=args.outdir)

### Incomplete Color Card 

In [ ]:
#.npz files containing target_matrix, source_matrix, and transformation_matrix will be saved to the output_directory file path

output_directory = "./test1"

In [ ]:
target_matrix, source_matrix, transformation_matrix, corrected_img = pcv.transform.correct_color(target_img, mask, source_img, mask, output_directory)

In [ ]:
transformation_matrix = pcv.transform.load_matrix("./test1/transformation_matrix.npz") #load in transformation_matrix

new_source = cv2.imread("VIS_SV_0_z1_h1_g0_e65_v500_376217_0.png") #read in new image for transformation

#apply transformation
corrected_img = pcv.transform.apply_transformation_matrix(source_img= new_source, target_img= target_img, transformation_matrix= transformation_matrix)

In [ ]:
target_img = cv2.imread("target_img.png")
source_img = cv2.imread("source_img.png")
mask = cv2.imread("mask.png", -1) # mask must be read in "as-is" include -1
#Since target_img and source_img have the same zoom and colorchecker position, the same mask can be used for both.

In [ ]:
# get color matrix of target and save
target_headers, target_matrix = pcv.transform.get_color_matrix(target_img, mask)
pcv.transform.save_matrix(target_matrix, "target.npz")

In [ ]:
#get color_matrix of source
source_headers, source_matrix = pcv.transform.get_color_matrix(source_img, mask)

In [ ]:
# matrix_a is a matrix of average rgb values for each color ship in source_img, matrix_m is a moore-penrose inverse matrix,
# matrix_b is a matrix of average rgb values for each color ship in source_img

matrix_a, matrix_m, matrix_b = pcv.transform.get_matrix_m(target_matrix= target_matrix, source_matrix= source_matrix)

In [ ]:
# deviance is the measure of how greatly the source image deviates from the target image's color space. 
# Two images of the same color space should have a deviance of ~0.
# transformation_matrix is a 9x9 matrix of transformation coefficients 

deviance, transformation_matrix = pcv.transform.calc_transformation_matrix(matrix_m, matrix_b)

In [ ]:
corrected_img = pcv.transform.apply_transformation_matrix(source_img= source_img, target_img= target_img, transformation_matrix= transformation_matrix)

In [ ]:
#This program illustrates how to create a gray-scale mask for use with plantcv.transform.correct_color.

from plantcv import plantcv as pcv
import cv2
import numpy as np

pcv.params.debug = "plot"

In [ ]:
# Read in img
img = cv2.imread("target_img.png") 
pcv.plot_image(img)

In [ ]:
#Using the pixel coordinate on the plotted image, designate a region of interest for an n x n pixel region in each color chip.

dimensions = [50,50]  #pixel ROI dimensions

chips = []
#Declare first row:
chips.append(pcv.roi.rectangle(img=img, x=1020, y = 1010, w = dimensions[0], h = dimensions[1])) #white
chips.append(pcv.roi.rectangle(img=img, x= 1150 , y= 1010 , w = dimensions[0], h = dimensions[1]))#blue
chips.append(pcv.roi.rectangle(img=img, x= 1280 , y= 1010 , w = dimensions[0], h = dimensions[1]))#orange
chips.append(pcv.roi.rectangle(img=img, x= 1420 , y= 1010 , w = dimensions[0], h = dimensions[1]))#brown
    


#declare y_shift
y_shift = 135

#declare number of total rows
row_total = 6

#declare all other rows
for i in range(1, row_total):
    chips.append(pcv.roi.rectangle(img=img, x=1020, y = 1010 + i*(y_shift), w = dimensions[0], h = dimensions[1]))
    chips.append(pcv.roi.rectangle(img=img, x= 1150 , y= 1010 + i*(y_shift), w = dimensions[0], h = dimensions[1]))
    chips.append(pcv.roi.rectangle(img=img, x= 1280 , y= 1010 + i*(y_shift), w = dimensions[0], h = dimensions[1]))
    chips.append(pcv.roi.rectangle(img=img, x= 1420 , y= 1010 + i*(y_shift), w = dimensions[0], h = dimensions[1]))

In [ ]:
#remove black and white
del chips[0]
del chips[19]

mask = np.zeros(shape=np.shape(img)[:2], dtype = np.uint8()) # create empty mask img.

print(mask)

In [ ]:
# draw contours for each region of interest and give them unique color values.

i=1
for chip in chips:
    print(chip)
    mask = cv2.drawContours(mask, chip[0], -1, (i*10), -1)
    i+=1


pcv.plot_image(mask, cmap="gray")

mask = mask*10  #multiply values in the mask for greater contrast. Exclude if designating have more than 25 color chips.

In [ ]:
np.unique(mask)

In [ ]:
cv2.imwrite("test_mask.png", mask) #write to file.

## Creating a pipeline with incomplete color data 

In [ ]:
from plantcv import plantcv as pcv
import cv2
import numpy as np
import matplotlib

In [ ]:
target_img = cv2.imread("target_img.png")
source_img = cv2.imread("source2_img.png")
target_mask = cv2.imread("test_mask.png", -1) # mask must be read in "as-is" include -1
source_mask = cv2.imread("mask2_img.png", -1) 

#.npz files containing target_matrix, source_matrix, and transformation_matrix will be saved to the output_directory file path
output_directory = "./test1"

In [ ]:
target_matrix, source_matrix, transformation_matrix, corrected_img = pcv.transform.correct_color(target_img, target_mask, source_img, source_mask, output_directory)

In [ ]:
transformation_matrix = pcv.transform.load_matrix("./test1/transformation_matrix.npz") #load in transformation_matrix

new_source = cv2.imread("VIS_SV_0_z1_h1_g0_e65_v500_376217_0.png") #read in new image for transformation

corrected_img = pcv.transform.apply_transformation_matrix(source_img= new_source, target_img= target_img, transformation_matrix= transformation_matrix) #apply transformation

## Checking the color card chips 

In [ ]:
from plantcv import plantcv as pcv
from plotnine import *
import numpy as np
import pandas as pd

pcv.transform.quick_color_check(source_matrix = s_matrix, target_matrix = t_matrix, num_chips = 24)